In [1]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

In [2]:
training_data=[]
count = 0
y=[]
x=[]
with open("processed_train_data.csv", "r") as file:
    content = csv.reader(file)

    next(content)
    for line in content:
        y.append(int(line[1]))#survived
        x1 = int(line[2])     #Pclass
        x2 = int(line[3])     #Sex
        x3 = float(line[4])   #Age---
        x4 = int(line[5])     #Sibsp
        x5 = int(line[6])     #Parch
        x6 = int(line[7])     #Ticket---
        x7 = float(line[8])   #Fare---
        x8 = int(line[9])     #Embarked
        # print(x1,x2,x3,x4,x5,x6,x7,x8)
        x.append(np.array([x1,x2,x3,x4,x5,x6,x7,x8]))
        
    # print(x)

In [3]:
mean = np.zeros(3)
std = np. zeros(3)

def Normalize (x,n=0):
    sum1 = 0
    sum2 = 0
    for i in range(len(x)):
        sum1 += x[i][n]
        sum2 += x[i][n]**2

    mean = sum1/(len(x))
    variance = sum2/(len(x)) - mean**2
    std = variance**(0.5)

    for i in range(len(x)):
        x[i][n] = (x[i][n] - mean)/std
    return mean, std
j=0
for i in [2,5,6]:
    mean[j], std[j] = Normalize(x, i)
    j+=1
    
# for line in x:
#     print(line.tolist())
print(mean, std)

[2.95497302e+01 2.98403925e+05 3.22042080e+01] [1.37640977e+01 6.55772060e+05 4.96655344e+01]


In [4]:
phi = 0
mu_0  = np.zeros(len(x[0]))
mu_1 = np.zeros(len(x[0]))
sigma = np.zeros((len(x[0]), len(x[0])))

for z in y:
    phi += z
phi /= len(y)

count =0
for a, b in zip(y,x):
    if a == 0:
        mu_0 += b
        count += 1
mu_0 /= count

count = 0
for a, b in zip(y,x):
    if a == 1:
        mu_1 += b
        count += 1
mu_1 /= count
print(phi)
print(mu_0)
print(mu_1)

for a,b in zip(y,x) :
    if a == 0:
        beta = b - mu_0
        sigma +=  np. outer(beta, beta)
        
    if a == 1:
        beta = b - mu_1
        sigma +=  np. outer(beta, beta)
sigma /= len(y)

print(sigma)


0.3838383838383838
[ 2.53187614  0.85245902  0.06966477  0.55373406  0.32969035  0.07593627
 -0.20308492  1.64116576]
[ 1.9502924   0.31871345 -0.11183029  0.47368421  0.46491228 -0.1218977
  0.32600474  1.35087719]
[[ 0.61823472 -0.02076308 -0.35843405  0.065502    0.03101451  0.21171198
  -0.38638755  0.076032  ]
 [-0.02076308  0.16084109  0.04317353 -0.07045917 -0.07740799  0.01196364
  -0.02031511  0.00862942]
 [-0.35843405  0.04317353  0.99220937 -0.36804921 -0.16906384 -0.10398777
   0.11610478 -0.0439185 ]
 [ 0.065502   -0.07045917 -0.36804921  1.21316274  0.37088479  0.04688563
   0.1859722   0.05751235]
 [ 0.03101451 -0.07740799 -0.16906384  0.37088479  0.64467452 -0.02071443
   0.1572711   0.03721608]
 [ 0.21171198  0.01196364 -0.10398777  0.04688563 -0.02071443  0.99074354
  -0.13208497  0.14151017]
 [-0.38638755 -0.02031511  0.11610478  0.1859722   0.1572711  -0.13208497
   0.93379335 -0.14792199]
 [ 0.076032    0.00862942 -0.0439185   0.05751235  0.03721608  0.14151017
  -

In [5]:
det_sigma = abs(np.linalg.det(sigma))
print(det_sigma)
sigma_inv = np.linalg.inv(sigma)
print(sigma_inv)

0.013389192416330797
[[ 2.95234425  0.23524785  0.86246627  0.01594651 -0.18146532 -0.40998981
   1.09854806  0.06087696]
 [ 0.23524785  6.67892037 -0.05958534  0.15020856  0.67652234 -0.10071431
   0.06960997 -0.14103226]
 [ 0.86246627 -0.05958534  1.45684593  0.34675566  0.11438648 -0.03204232
   0.07927652 -0.0145493 ]
 [ 0.01594651  0.15020856  0.34675566  1.12188431 -0.48570188 -0.04518829
  -0.19565257 -0.09079781]
 [-0.18146532  0.67652234  0.11438648 -0.48570188  2.04399143  0.08603083
  -0.33469495 -0.15624744]
 [-0.40998981 -0.10071431 -0.03204232 -0.04518829  0.08603083  1.12358027
  -0.04923876 -0.21984415]
 [ 1.09854806  0.06960997  0.07927652 -0.19565257 -0.33469495 -0.04923876
   1.65530246  0.31445837]
 [ 0.06087696 -0.14103226 -0.0145493  -0.09079781 -0.15624744 -0.21984415
   0.31445837  1.750057  ]]


In [6]:
def prediction (x):
    factor = (x- mu_0).T @ sigma_inv
    factor = factor @ (x - mu_0)
    factor /= 2
    p_0 = np.exp(-np.clip(factor, -500, 500))
    p_0 /= ((2*np.pi)**(len(mu_0)/2) * det_sigma**(0.5))

    factor = (x- mu_1).T @ sigma_inv
    factor = factor @ (x - mu_1)
    factor /= 2
    p_1 = np.exp(-np.clip(factor, -500, 500))
    p_1 /= ((2*np.pi)**(len(mu_1)/2) * det_sigma**(0.5))

    if phi * p_1 > (1-phi)* p_0:
        return 1
    else:
        return 0

In [7]:
positive = 0
for a,b in zip(y,x):
    if prediction(b) == a:
        positive += 1

print("positives =",positive)
print("percentage =", 100*positive/len(x),"%")

positives = 713
percentage = 80.02244668911335 %


In [8]:
p=[]
t=[]
with open("processed_test_data.csv", "r") as file:
    content = csv.reader(file)

    next(content)
    for line in content:
        p.append(int(line[0]))#passengerID
        x1 = int(line[1])     #Pclass
        x2 = int(line[2])     #Sex
        x3 = float(line[3])   #Age ---
        x4 = int(line[4])     #Sibsp
        x5 = int(line[5])     #Parch
        x6 = float(line[6])     #Ticket ---
        x7 = float(line[7])   #Fare ---
        x8 = int(line[8])     #Embarked
        # print(x1,x2,x3,x4,x5,x6,x7,x8)
        t.append(np.array([x1,x2,x3,x4,x5,x6,x7,x8]))
        
output = [['PassengerId', 'Survived']]
for i in range(len(p)):
    z = t[i]
    z[2] = (x[i][2]- mean[0])/(std[0])
    z[5] = (x[i][5]- mean[1])/(std[1])
    z[6] = (x[i][6]- mean[2])/(std[2])

    output.append([p[i], prediction (z)])
# print(output)

In [9]:
with open("GDA_output.csv", "w") as file:
    writer = csv.writer(file)
    writer.writerows(output)